In [7]:
import yfinance as yf
import numpy as np
from qiskit.result import QuasiDistribution
from qiskit_aer.primitives import Sampler
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_optimization.algorithms import MinimumEigenOptimizer

stocks = ["RELIANCE.NS", "HDFCBANK.NS", "INFY.NS", "TCS.NS", "ICICIBANK.NS", 
                   "HINDUNILVR.NS", "KOTAKBANK.NS", "LT.NS", "SBIN.NS", "AXISBANK.NS", 
                   "BAJFINANCE.NS", "BHARTIARTL.NS", "MARUTI.NS", "SUNPHARMA.NS", 
                   "TITAN.NS", "ULTRACEMCO.NS", "TATASTEEL.NS", "ASIANPAINT.NS", 
                   ]

# Download historical stock data for 3 years
df = yf.download(stocks, start="2021-01-01", end="2024-01-01")
df = df['Close']

# Step 2: Calculate Returns & Covariance Matrix
returns = df.pct_change().dropna()
expected_returns = returns.mean()
cov_matrix = returns.cov()

# Step 3: Construct QUBO Matrix
assets = len(expected_returns)
penalty = 1 / np.mean(abs(cov_matrix.values))  # Adjust for risk-return balance


[*********************100%***********************]  18 of 18 completed


In [8]:
tickers = ["RELIANCE.NS", "HDFCBANK.NS", "INFY.NS", "TCS.NS", "ICICIBANK.NS", 
                   "HINDUNILVR.NS", "KOTAKBANK.NS", "LT.NS", "SBIN.NS", "AXISBANK.NS", 
                   "BAJFINANCE.NS", "BHARTIARTL.NS", "MARUTI.NS", "SUNPHARMA.NS", 
                   "TITAN.NS", "ULTRACEMCO.NS", "TATASTEEL.NS", "ASIANPAINT.NS", 
                   ]
expected_returns = expected_returns[tickers].values
cov_matrix = cov_matrix.loc[tickers, tickers].values

print(expected_returns)
print(cov_matrix)

[5.83678265e-04 3.82466985e-04 4.80228066e-04 4.90240308e-04
 1.00856476e-03 2.89893828e-04 5.16866742e-05 1.51728721e-03
 1.33380363e-03 9.13062507e-04 6.36177977e-04 1.08379832e-03
 5.41478872e-04 1.14902010e-03 1.30495786e-03 1.06359204e-03
 2.88345226e-03 4.19013952e-04]
[[2.22596480e-04 7.81369430e-05 6.99376691e-05 5.78639285e-05
  8.35737272e-05 4.01980767e-05 7.73487595e-05 7.88472515e-05
  9.90143748e-05 8.52243026e-05 1.10816016e-04 6.37819789e-05
  5.97595977e-05 4.05852519e-05 7.63820734e-05 7.33749421e-05
  1.00301246e-04 4.99162481e-05]
 [7.81369430e-05 2.00683723e-04 6.88004136e-05 5.55732192e-05
  1.20675017e-04 3.98975100e-05 1.17869029e-04 8.71171013e-05
  1.12402253e-04 1.00214099e-04 1.19680307e-04 6.67055393e-05
  6.44246709e-05 4.27358304e-05 7.18449209e-05 8.01685255e-05
  5.57909735e-05 4.78851270e-05]
 [6.99376691e-05 6.88004136e-05 2.31265230e-04 1.38577454e-04
  6.66238201e-05 4.80696993e-05 4.75061944e-05 8.53428421e-05
  5.67929555e-05 5.45654253e-05 8.7251

In [16]:
# q = 0.5  # set risk factor
budget = assets  # set budget

portfolio = PortfolioOptimization(
    expected_returns=expected_returns, covariances=cov_matrix, risk_factor=0.2, budget=budget
)
qp = portfolio.to_quadratic_program()

In [17]:
def print_result(result):
    selection = result.x
    value = result.fval
    print("Optimal: selection {}, value {:.4f}".format(selection, value))

    eigenstate = result.min_eigen_solver_result.eigenstate
    probabilities = (
        eigenstate.binary_probabilities()
        if isinstance(eigenstate, QuasiDistribution)
        else {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}
    )
    print("\n----------------- Full result ---------------------")
    print("selection\tvalue\t\tprobability")
    print("---------------------------------------------------")
    probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

    for k, v in probabilities:
        x = np.array([int(i) for i in list(reversed(k))])
        value = portfolio.to_quadratic_program().objective.evaluate(x)
        print("%10s\t%.4f\t\t%.4f" % (x, value, v))

In [18]:
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), reps=10)
qaoa = MinimumEigenOptimizer(qaoa_mes)
result = qaoa.solve(qp)

print(result)
print("Optimal Portfolio Selection:", result.x)
print("Optimal Portfolio Value:", result.fval)

print("\n\n")
print_result(result)

fval=-0.010478885950076428, x_0=1.0, x_1=1.0, x_2=1.0, x_3=1.0, x_4=1.0, x_5=1.0, x_6=1.0, x_7=1.0, x_8=1.0, x_9=1.0, x_10=1.0, x_11=1.0, x_12=1.0, x_13=1.0, x_14=1.0, x_15=1.0, x_16=1.0, x_17=1.0, status=SUCCESS
Optimal Portfolio Selection: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Optimal Portfolio Value: -0.010478885950076428



Optimal: selection [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.], value -0.0105

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]	-0.0105		0.2002
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]	-0.0104		0.0049
[1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1]	-0.0106		0.0039
[1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1]	-0.0098		0.0039
[1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]	-0.0105		0.0039
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]	0.0000		0.0029
[1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1]	-0.0081		0.0029
[1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1]	-0.0090		0.00